# Einmalige Konvertierung von GeoJSON zu GeoPackage
Lärmkarte (Tag, 2022) für Brandenburg mit Bounding Box für Brandenburg an der Havel als GeoJSON vom Server herunterladen und lokal speichern (~ 630 MB):
```
curl "https://inspire.brandenburg.de/services/laerm_wfs?service=WFS&version=2.0.0&request=GetFeature&typeNames=app:bb_strasse_lden2022&bbox=12.3120236786,52.2938432979,12.7562682548,52.5594777244,EPSG:4326&outputFormat=application%2Fgeo%2Bjson" -o laerm.geojson
```
Einmalig zu Geopackage konvertieren mit diesem Skript (entkommentieren):

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import os
os.environ["OGR_GEOJSON_MAX_OBJ_SIZE"] = "1000"  # in MB

# Load huge GeoJSON file
#gdf = gpd.read_file("data/laerm.geojson")
# Convert GeoJSON to GeoPackage for performance
#geopackage_path = "data/laerm.gpkg"
#gdf.to_file(geopackage_path, driver="GPKG")
#print(f"Konvertierung abgeschlossen: {geopackage_path}")


# Eigentliche Analyse


In [ ]:
from shapely import union_all
import geopandas as gpd
import pandas as pd
import folium
import branca.colormap as cm
from shapely.geometry import box
from folium.plugins import HeatMap

# -------------------------
# 1. Lärmdaten laden & zuschneiden
# -------------------------
gdf = gpd.read_file("data/laerm.gpkg")

# Bounding Box "Brandenburg an der Havel" ausschneiden
bbox = box(12.3120236786, 52.2938432979, 12.7562682548, 52.5594777244)
gdf = gdf.clip(bbox)

# Lärmwert-Spalte bestimmen
num_cols = gdf.select_dtypes(include="number").columns
value_column = num_cols[0] if len(num_cols) > 0 else None
print(f"Farbschema basiert auf Spalte: {value_column}")

# Colormap
min_val = gdf[value_column].min()
max_val = gdf[value_column].max()
colormap = cm.LinearColormap(colors=["green", "yellow", "red"], vmin=min_val, vmax=max_val)
colormap.caption = "Lärmpegel (LDEN in dB)"

# Kartenmittelpunkt berechnen
centroid = union_all(gdf.geometry).centroid
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=14, tiles="CartoDB positron")

# Lärmpolygone darstellen
def style_function(feature):
    value = feature["properties"][value_column]
    return {
        "fillColor": colormap(value),
        "color": "black",
        "weight": 0.3,
        "fillOpacity": 0.6
    }

folium.GeoJson(
    gdf,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=[value_column])
).add_to(m)

colormap.add_to(m)

# -------------------------
# 2. Adresspunkte als kleine Marker
# -------------------------
df_adressen = pd.read_csv("data/adressen_geocoded.csv")
df_adressen = df_adressen.dropna(subset=["lat", "lon"])

# Adressen in GeoDataFrame umwandeln
gdf_adressen = gpd.GeoDataFrame(
    df_adressen,
    geometry=gpd.points_from_xy(df_adressen["lon"], df_adressen["lat"]),
    crs="EPSG:4326"
)

# Adressen mit Lärmpolygonen verschneiden
value_columns = ["isov1", "isov2", "geometry"]
gdf_adressen = gpd.sjoin(gdf_adressen, gdf[value_columns], how="left", predicate="within")

for _, row in gdf_adressen.iterrows():
    tooltip = f"{row['Straßenname']} {row['Hsnr']}{row['HsnrZus'] or ''} – {round(row[value_column], 1)} dB"
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=3,
        color="blue",
        fill=True,
        fill_opacity=0.7,
        tooltip=tooltip
    ).add_to(m)


# -------------------------
# 4. Karte anzeigen
# -------------------------
m


In [ ]:
# Rename index_right to Laerm_index
gdf_adressen.rename(columns={"index_right": "Laerm_index_tag"}, inplace=True)

# Rename isov1 and isov2 to Laerm_tag_von and Laerm_tag_bis
gdf_adressen.rename(columns={"isov1": "Laerm_tag_von", "isov2": "Laerm_tag_bis"}, inplace=True)

# Create a CSV file with the address and noise data from GDF
gdf_adressen.to_csv("data/adressen_mit_laerm.csv", index=False)


gdf_adressen.describe()
